In [2]:
import sys
sys.path.insert(0, '/Users/annabroido/Dropbox/Research/LRTAnalysis/SFAnalysis/analysis/')
import numpy as np
import pandas as pd
import importfiles as im

In [3]:
# import data
df = pd.read_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/SFAnalysis/local/newanalysis.p')
df = df.query("Domain != 'Economic'")
df = df.query("alpha != ''")
# NOTE THAT 'IN' MEAN IN TO THE ALT DISTRIBUTION, WHCIH IS NOT WHAT MATHEMATICA DOES

In [70]:
np.unique(df2.fp_gml.values).shape
#np.sum([gml in cat.fp_gml.values for gml in np.unique(df.fp_gml)])

(927,)

In [35]:
df2[df2.fp_gml == '/Volumes/Durodon/gmls/Technological/Communication/n4/Route_Views_AS_graphs_1997-1998_as19971210_Technological_Communication_n4.gml']

,Domain,Subdomain,num_edges,Graph_order,Weighted,Directed,Bipartite,Multigraph,Multiplex,Component,...,n,alpha,xmin,ntail,Lpl,ppl,dexp,dln,dstrexp,dplwc


In [72]:
# df2 = pd.read_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/SFAnalysis/analysis/analysis.p')
# cat = pd.read_pickle('newgmlcatalog.p')
# cat2 = pd.read_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/SFAnalysis/analysis/gmlcatalog.p')
len(cat2)

2009

In [38]:
def getcount(decision, direction, df=df):
    directiondir = {'in':'-1', 'out':'1', 'unk':'0', 'fail':'2'}
    query = '%s == %s' %(decision, directiondir[direction])
    num = len(np.asarray(df.query(query)[decision], dtype=int))
    return num

In [39]:
# plot some stuff
expin = getcount('dexp', 'in')
expout = getcount('dexp', 'out')
expunk = getcount('dexp', 'unk')
expfail = getcount('dexp', 'fail')
lnin = getcount('dln', 'in')
lnout = getcount('dln', 'out')
lnunk = getcount('dln', 'unk')
lnfail = getcount('dln', 'fail')
strexpin = getcount('dstrexp', 'in')
strexpout = getcount('dstrexp', 'out')
strexpunk = getcount('dstrexp', 'unk')
strexpfail = getcount('dstrexp', 'fail')
plwcin = getcount('dplwc', 'in')
plwcout = getcount('dplwc', 'out')
plwcunk = getcount('dplwc', 'unk')
# order = "LN", "Exp", "Str Exp", "PLwC"
incounts = [expin, lnin, strexpin, plwcin]
outcounts = [expout, lnout, strexpout, plwcout]
unkcounts = [expunk, lnunk, strexpunk, plwcunk]
print incounts
print outcounts
print unkcounts
print [expfail, lnfail, strexpfail]

[767, 1061, 995, 1321]
[2050, 399, 1822, 0]
[528, 2100, 743, 2340]
[316, 101, 101]


In [40]:
unique_datasets = np.unique(df.fp_gml)
decisions = ['dexp','dln', 'dstrexp', 'dplwc']
allcounts = np.zeros((3,4))
# each column of counts (stack in, on, and unk) adds up to len(rows) unless some fail
for i, dataset in enumerate(unique_datasets):
    query = "fp_gml == '%s'" %dataset
    rows = df.query(query)
    counts = np.zeros((3,4)) # rows are in, out, and unk 
    for i, decision in enumerate(decisions):
        counts[0,i] = getcount(decision, 'out', rows)
        counts[1,i] = getcount(decision, 'unk', rows)
        counts[2,i] = getcount(decision, 'in', rows)
        counts[1,i]+= getcount(decision,'fail',rows)
    counts=counts/len(rows) # normalize each dataset
    allcounts += counts

In [41]:
# unique_datasets = np.unique(df.fp_gml)
# decisions = ['dexp','dln', 'dstrexp', 'dplwc']
# allcounts = np.zeros((3,4))

In [42]:
lrtproportions = 100*allcounts/len(unique_datasets)

In [43]:
countdf = pd.DataFrame(index=['Exponential', 'Log-normal', 'Weibull', 'PLwC'], columns=['M_PL', 'Inconclusive', 'M_Alt'])
countdf.loc['Exponential'] = lrtproportions[:,0]
countdf.loc['Log-normal'] = lrtproportions[:,1]
countdf.loc['Weibull'] = lrtproportions[:,2]
countdf.loc['PLwC'] = lrtproportions[:,3]
countdf

,M_PL,Inconclusive,M_Alt
Exponential,55.6829,23.0009,21.2984
Log-normal,7.65119,66.768,25.563
Weibull,45.7112,29.5779,24.6931
PLwC,0,68.9467,31.0355


In [13]:
# doms = np.unique(df.Domain)
# # number of graphs and mean alpha value in each domain
# counts = np.zeros_like(doms)
# means = np.zeros_like(doms)
# counts_plaus = np.zeros_like(doms)
# means_plaus = np.zeros_like(doms)
# counts_plaustail = np.zeros_like(doms)
# means_plaustail = np.zeros_like(doms)
# for i,dom in enumerate(doms):
#     query = "Domain == '%s'" %dom
#     subdf = df.query(query)
#     counts[i] = len(subdf)
#     means[i] = format(np.mean(subdf.alpha), '.2f')
#     subdf_plaus = subdf.query("ppl>0.1")
#     counts_plaus[i] = len(subdf_plaus)
#     means_plaus[i] = format(np.mean(subdf_plaus.alpha), '.2f')
#     subdf_plaustail = subdf_plaus.query("ntail>=50")
#     counts_plaustail[i] = len(subdf_plaustail)
#     means_plaustail[i] = format(np.mean(subdf_plaustail.alpha), '.2f')

In [14]:
# print doms
# print counts_plaustail
# print means_plaustail

In [4]:
def test_strong_noplwc(rows):
    ''' neither have plwc, the weaker one has no range on alpha'''
    A1 = False # strongnoplwc
    A2 = False # no plwc or range
    n = len(rows)
    np = 0 #noplwc
    npnr = 0 #noplwcnorange
    for ind, row in rows.iterrows():
        if row.ppl>0.1 and row.ntail >= 50 and row.dexp >-1 and row.dln>-1  and row.dstrexp >-1:
            npnr += 1
            if row.alpha < 3 and row.alpha > 2:
                np += 1
    if np >= n/2.:
        A1 = True
    if npnr>= n/2.:
        A2 = True
    return (A1, A2)

In [5]:
unique_datasets = np.unique(df.fp_gml)
althyps = pd.DataFrame(columns = ["No_PLwC", "No_PLwC_No_Range"], index=unique_datasets )
for i, dataset in enumerate(unique_datasets):
    query = "fp_gml == '%s'" %dataset
    rows = df.query(query)
    [A1, A2] = test_strong_noplwc(rows)
    althyps.loc[dataset]['No_PLwC'] = A1
    althyps.loc[dataset]['No_PLwC_No_Range'] = A2

In [63]:
# def test_strong(rows):
#     S1 = False # strongest
#     S2 = False # strong
#     SA = False
#     n = len(rows)
#     strong = 0
#     strong_alone = 0
#     for ind, row in rows.iterrows():
#         if row.ppl>0.1 and row.ntail >= 50 and row.alpha < 3 and row.alpha > 2:
#             strong_alone += 1
#             if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1  and row.dplwc >-1:
#                 strong += 1
#     if strong_alone >= 9.*n/10:
#         SA = True
#     if strong >= n/2.:
#         S2 = True
#     if SA == True and strong >= 95.*n/100:
#         S1 = True
#     return (S1, S2)

def test_strongest_noplwc(rows):
    ''' neither have plwc, the weaker one has no range on alpha'''
    A1 = False # noplwcstrongest
    A2 = False # no plwc or range strongest
    n = len(rows)
    nps = 0 #strongestnoplwc
    npnrs = 0 #noplwcnorange
    alts = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1 and row.ntail >= 50:
            npnrs += 1
            if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1:
                alts+= 1
            if row.alpha < 3 and row.alpha > 2:
                nps += 1
    if alts >= 95.*n/100. and nps >= 90.*n/100.:
        A1 = True
    if alts >= 95.*n/100. and npnrs >= 95.*n/100.:
        A2 = True
    return (A1, A2)

def test_weak_noplwc(rows):
    W = False 
    West = False 
    SW = False
    n = len(rows)
    weak = 0
    weakest = 0
    sweak = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1:
            weakest += 1
            if row.ntail>=50:
                weak += 1
        if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1:
            sweak += 1
    if weak >= n/2.:
        W = True
    if weakest >= n/2.:
        West = True
    if sweak >= n/2.:
        SW = True
    return (W, West, SW)

In [45]:
def test_strong_any(rows):
    ''' just need a minimum of one dataset'''
    S = False # strongest/strong
    n = len(rows)
    strong_alone = 0 #noplwc
    strong = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1 and row.ntail >= 50 and row.alpha <3 and row.alpha > 2:
            if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1 and row.dplwc >-1:
                strong += 1
    if strong>= 1:
        S = True
    return (S)

def test_weak_any(rows):
    ''' just need a minimum of one dataset'''
    W = False 
    West = False 
    SW = False
    n = len(rows)
    weak = 0
    weakest = 0
    sweak = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1:
            weakest += 1
            if row.ntail>=50:
                weak += 1
        if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1  and row.dplwc >-1:
            sweak += 1
    if weak >= 1:
        W = True
    if weakest >= 1:
        West = True
    if sweak >= 1:
        SW = True
    return (W, West, SW)

In [46]:
def test_strong_any_noplwc(rows):
    ''' just need a minimum of one dataset, no plwc'''
    S = False # strongest/strong
    n = len(rows)
    strong_alone = 0 #noplwc
    strong = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1 and row.ntail >= 50 and row.alpha <3 and row.alpha > 2:
            if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1:
                strong += 1
    if strong>= 1:
        S = True
    return (S)

In [47]:
def test_strong_any_nobounds(rows):
    ''' just need a minimum of one dataset, no plwc, nobounds'''
    S = False # strongest/strong
    n = len(rows)
    strong_alone = 0 #noplwc
    strong = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1 and row.ntail >= 50:
            if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1:
                strong += 1
    if strong>= 1:
        S = True
    return (S)

In [48]:
def test_strong(rows):
    S1 = False # strongest
    S2 = False # strong
    SA = False
    n = len(rows)
    strong = 0
    strong_alone = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1 and row.ntail >= 50 and row.alpha < 3 and row.alpha > 2:
            strong_alone += 1
            if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1  and row.dplwc >-1:
                strong += 1
    if strong_alone >= 9.*n/10:
        SA = True
    if strong >= n/2.:
        S2 = True
    if SA == True and strong >= 95.*n/100:
        S1 = True
    return (S1, S2)
            
def test_weak(rows):
    W = False 
    West = False 
    SW = False
    n = len(rows)
    weak = 0
    weakest = 0
    sweak = 0
    for ind, row in rows.iterrows():
        if row.ppl>0.1:
            weakest += 1
            if row.ntail>=50:
                weak += 1
        if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1  and row.dplwc >-1:
            sweak += 1
    if weak >= n/2.:
        W = True
    if weakest >= n/2.:
        West = True
    if sweak >= n/2.:
        SW = True
    return (W, West, SW)
          

In [67]:
# test diff hypotheses
unique_datasets = np.unique(df.fp_gml)
hyps = pd.DataFrame(columns = ['Strongest', "Strong", "Weak", "Weakest", "Super_Weak", "Strong_No_PLwC", "Strong_No_PLwC_No_Range","Strongest_No_PLwC", "No_PLwC_No_Range_Strongest", "Weak_PLwC", "Weakest_PLwC", "Super_Weak_PLwC", "Strong_Any", "Strong_Any_noplwc", "Strong_Any_nobounds","Weak_Any", "Weakest_Any", "Super_Weak_Any", "Domain", "Subdomain", "median_alpha", "n", "median_ntail"], index=unique_datasets )
for i, dataset in enumerate(unique_datasets):
    query = "fp_gml == '%s'" %dataset
    rows = df.query(query)
    [S1, S2] = test_strong(rows)
    hyps.loc[dataset]['Strongest'] = S1
    hyps.loc[dataset]['Strong'] = S2
    [weak,weakest,superweak] = test_weak(rows)
    hyps.loc[dataset]['Weak'] = weak
    hyps.loc[dataset]['Weakest'] = weakest
    hyps.loc[dataset]['Super_Weak'] = superweak
    [A1, A2] = test_strong_noplwc(rows)
    hyps.loc[dataset]['Strong_No_PLwC'] = A1
    hyps.loc[dataset]['No_PLwC_No_Range'] = A2
    [A1S, A2S] = test_strongest_noplwc(rows)
    hyps.loc[dataset]['Strongest_No_PLwC']= A1S
    hyps.loc[dataset]['No_PLwC_No_Range_Strongest'] = A2S
    [W, West, SW] = test_weak_noplwc(rows)
    hyps.loc[dataset]['Weak_PLwC'] = W
    hyps.loc[dataset]['Weakest_PLwC'] = West
    hyps.loc[dataset]['Super_Weak_PLwC'] = SW
    S = test_strong_any(rows)
    hyps.loc[dataset]['Strong_Any'] = S
    S = test_strong_any_noplwc(rows)
    hyps.loc[dataset]['Strong_Any_noplwc'] = S
    S = test_strong_any_nobounds(rows)
    hyps.loc[dataset]['Strong_Any_nobounds'] = S
    [W, West, SW] = test_weak_any(rows)
    hyps.loc[dataset]['Weak_Any'] = W
    hyps.loc[dataset]['Weakest_Any'] = West
    hyps.loc[dataset]['Super_Weak_Any'] = SW
    hyps.loc[dataset]['Domain'] = rows.Domain[0]
    hyps.loc[dataset]['Subdomain'] = rows.Subdomain[0]
    hyps.loc[dataset]['median_alpha'] = np.median(rows.alpha)
    hyps.loc[dataset]['n'] = np.max(rows.n)
    hyps.loc[dataset]['median_ntail'] = np.median(rows.ntail)
    
    

In [70]:
n = float(len(hyps))
Strongest = np.sum(hyps.Strongest)
Strong = np.sum(hyps.Strong)
Weak = np.sum(hyps.Weak)
Weakest = np.sum(hyps.Weakest)
Super_Weak = np.sum(hyps.Super_Weak)
Strong_No_PLwC = np.sum(hyps.Strong_No_PLwC)
Strong_No_PLwC_No_Range = np.sum(hyps.Strong_No_PLwC_No_Range)
No_PLwC_Strongest = np.sum(hyps.Strongest_No_PLwC)
No_PLwC_No_Range_Strongest = np.sum(hyps.No_PLwC_No_Range_Strongest)
Weak_NoPLwC = np.sum(hyps.Weak_PLwC)
Weakest_NoPLwC = np.sum(hyps.Weakest_PLwC)
Super_Weak_NoPLwC = np.sum(hyps.Super_Weak_PLwC)
Strong_Any = np.sum(hyps.Strong_Any)
Strong_Any_noplwc = np.sum(hyps.Strong_Any_noplwc)
Strong_Any_nobounds = np.sum(hyps.Strong_Any_nobounds)
Weak_Any = np.sum(hyps.Weak_Any)
Weakest_Any = np.sum(hyps.Weakest_Any)
Super_Weak_Any = np.sum(hyps.Super_Weak_Any)
print "Strongest = %s" %(Strongest/n)
print "Strong = %s" %(Strong/n)
print "Weak = %s" %(Weak/n)
print "Weakest = %s" %(Weakest/n)
print "Super Weak = %s" %(Super_Weak/n)
print "No PLwC Strong = %s" %(Strong_No_PLwC/n)
print "No PLwC No Range = %s" %(No_PLwC_No_Range/n)
print "No PLwC Strongest = %s" %(No_PLwC_Strongest/n)
print "No PLwC No Range = %s" %(No_PLwC_No_Range_Strongest/n)
print "Weak No PLwC = %s" %(Weak_NoPLwC/n)
print "Weakest No PLwC = %s" %(Weakest_NoPLwC/n)
print "Super-Weak No PLwC = %s" %(Super_Weak_NoPLwC/n)
print "Strong Any = %s" %(Strong_Any/n)
print "Strong Any No PLWC = %s" %(Strong_Any_noplwc/n)
print "Strong Any No Bounds = %s" %(Strong_Any_nobounds/n)
print "Weak Any = %s" %(Weak_Any/n)
print "Weakest Any = %s" %(Weakest_Any/n)
print "Super-Weak Any = %s" %(Super_Weak_Any/n)
print "Number of Datsets = %s" %int(n)

Strongest = 0.0377562028047
Strong = 0.105717367853
Weak = 0.24163969795
Weakest = 0.333333333333
Super Weak = 0.522114347357
No PLwC Strong = 0.114347357066
No PLwC No Range = 0.228694714132
No PLwC Strongest = 0.0442286947141
No PLwC No Range = 0.1380798274
Weak No PLwC = 0.24163969795
Weakest No PLwC = 0.333333333333
Super-Weak No PLwC = 0.567421790723
Strong Any = 0.188781014024
Strong Any No PLWC = 0.196332254585
Strong Any No Bounds = 0.314994606257
Weak Any = 0.327939590076
Weakest Any = 0.425026968716
Super-Weak Any = 0.541531823085
Number of Datsets = 927


In [58]:
dataset=unique_datasets[0]
query = "fp_gml == '%s'" %dataset
rows = df.query(query)
rows
A1 = False # noplwcstrongest
A2 = False # no plwc or range strongest
n = len(rows)
nps = 0 #strongestnoplwc
npnrs = 0 #noplwcnorange
alts = 0
for ind, row in rows.iterrows():
    if row.ppl>0.1 and row.ntail >= 50:
        npnrs += 1
        if row.dexp >-1 and row.dln>-1  and row.dstrexp >-1:
            alts+= 1
        if row.alpha < 3 and row.alpha > 2:
            nps += 1
if alts >= 95.*n/10. and nps >= 90.*n/100.:
    A1 = True
if alts >= 95.*n/10. and npnrs >= 95.*n/100.:
    A2 = True


False

In [11]:
# dsfp = '/Users/annabroido/Dropbox/Research/LRTAnalysis/degreesequences/' # degree sequences

# hyps['median_meandeg'] = ''

# unique_datasets = np.unique(df.fp_gml)
# for i, dataset in enumerate(unique_datasets):
#     query = "fp_gml == '%s'" %dataset
#     rows = df.query(query)
#     means = np.zeros(len(rows))
#     for r, row in enumerate(rows.iterrows()):
#         name = row[1].name
#         fp = dsfp + name
#         x = im.readdata(fp)
#         means[r] = np.mean(x)
#     hyps.loc[dataset]['median_meandeg'] = np.median(means)

In [71]:
hyps.to_pickle('/Users/annabroido/Dropbox/Research/LRTAnalysis/SFAnalysis/analysis/newhyps.p')
# hyps.to_csv('/Users/annabroido/Dropbox/Research/LRTAnalysis/SFAnalysis/analysis/newhyps.csv')

In [30]:
len(hyps)

927

In [78]:
df.query()

,Domain,Subdomain,num_edges,Graph_order,Weighted,Directed,Bipartite,Multigraph,Multiplex,Component,...,n,alpha,xmin,ntail,Lpl,ppl,dexp,dln,dstrexp,dplwc
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexroad_category_multigraphsimplifieddistribution.txt,Transportation,Roads,1078246,6,0,0,0,simplified,sub_road_category,entire,...,,6.3,3,456034,-291964,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexroad_category_multigraphsimplified_largestcompdistribution.txt,Transportation,Roads,1072638,6,0,0,0,simplified,sub_road_category,largest,...,,6.3,3,455270,-291717,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplextravel_time_multigraphsimplifieddistribution.txt,Transportation,Roads,1078246,6,0,0,0,simplified,sub_travel_time,entire,...,,6.3,3,456034,-291964,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplextravel_time_multigraphsimplified_largestcompdistribution.txt,Transportation,Roads,1072638,6,0,0,0,simplified,sub_travel_time,largest,...,,6.3,3,455270,-291717,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexspatial_distance_in_meters_multigraphsimplifieddistribution.txt,Transportation,Roads,1078246,6,0,0,0,simplified,sub_spatial_distance_in_meters,entire,...,,6.3,3,456034,-291964,0,-1,-1,-1,-1
United_States_Road_Networks_PA_Transportation_Roads_n6.gml_multiplexspatial_distance_in_meters_multigraphsimplified_largestcompdistribution.txt,Transportation,Roads,1072638,6,0,0,0,simplified,sub_spatial_distance_in_meters,largest,...,,6.3,3,455270,-291717,0,-1,-1,-1,-1


In [27]:
subhyps = hyps.query("Domain=='Social'")
n = float(len(subhyps))
Strongest = np.sum(subhyps.Strongest)
Strong = np.sum(subhyps.Strong)
Weak = np.sum(subhyps.Weak)
Weakest = np.sum(subhyps.Weakest)
Super_Weak = np.sum(subhyps.Super_Weak)
No_PLwC = np.sum(subhyps.No_PLwC)
No_PLwC_No_Range = np.sum(subhyps.No_PLwC_No_Range)
No_PLwC_Strongest = np.sum(subhyps.No_PLwC_Strongest)
No_PLwC_No_Range_Strongest = np.sum(subhyps.No_PLwC_No_Range_Strongest)
Strong_Any = np.sum(subhyps.Strong_Any)
Strong_Any_noplwc = np.sum(subhyps.Strong_Any_noplwc)
Strong_Any_nobounds = np.sum(subhyps.Strong_Any_nobounds)
Weak_Any = np.sum(subhyps.Weak_Any)
Weakest_Any = np.sum(subhyps.Weakest_Any)
Super_Weak_Any = np.sum(subhyps.Super_Weak_Any)
print "Strongest = %s" %(Strongest/n)
print "Strong = %s" %(Strong/n)
print "Weak = %s" %(Weak/n)
print "Weakest = %s" %(Weakest/n)
print "Super Weak = %s" %(Super_Weak/n)
print "No PLwC = %s" %(No_PLwC/n)
print "No PLwC No Range = %s" %(No_PLwC_No_Range/n)
print "No PLwC Strongest = %s" %(No_PLwC_Strongest/n)
print "No PLwC No Range = %s" %(No_PLwC_No_Range_Strongest/n)
print "Strong Any = %s" %(Strong_Any/n)
print "Strong Any No PLWC = %s" %(Strong_Any_noplwc/n)
print "Strong Any No Bounds = %s" %(Strong_Any_nobounds/n)
print "Weak Any = %s" %(Weak_Any/n)
print "Weakest Any = %s" %(Weakest_Any/n)
print "Super-Weak Any = %s" %(Super_Weak_Any/n)
print "Number of Datsets = %s" %int(n)

Strongest = 0.0
Strong = 0.0
Weak = 0.551724137931
Weakest = 0.696551724138
Super Weak = 0.710344827586
No PLwC = 0.0
No PLwC No Range = 0.48275862069
No PLwC Strongest = 0.0
No PLwC No Range = 0.406896551724
Strong Any = 0.0137931034483
Strong Any No PLWC = 0.0137931034483
Strong Any No Bounds = 0.503448275862
Weak Any = 0.572413793103
Weakest Any = 0.710344827586
Super-Weak Any = 0.71724137931
Number of Datsets = 145
